# Fetch raw data
MNSIT is an org that put out 70K pictures of handwritten numerical digits. It's understood in the ML community that saying MNIST refers to the dataset, because this is an industry-standard beginner's exercise, like the "Hello, World" of ML.

If we successfully download the images, the keys will look like:
dict_keys(['data', 'target', 'feature_names', 'DESCR', 'details','categories', 'url'])

In [ ]:
from sklearn.datasets import fetch_openml # this is tool for fetching named datasets
mnist = fetch_openml('mnist_784', version=1) #need to download 15 megabytes of compressed images (55 mb) once, then scikit will cache it automatically
mnist.keys()